In [7]:
def relative_id_nosort(line_target,line_ref):
    N_target = len(line_target)
    id_target = []
    N_ref = len(line_ref)
    for ip in range(0,N_target):
        val_target = line_target[ip]
        for i in range(0,N_ref-1):
            if (line_ref[i]<=val_target and line_ref[i+1]>val_target):
                id_target.append(i)
                break
        if (len(id_target)<ip+1): 
            id_target.append(-1)    
        if (len(id_target)!=ip+1): 
            print('error')
            print(len(id_target),ip)
    return id_target 


In [8]:
import h5py
import numpy as np

def read_model_file_horizontal(grid_fname,model_fname,Nstep,stlat,edlat,stlon,edlon,Nlat,Nlon,all_dep):
    R_earth = 6371.0

    fmodel  = h5py.File(model_fname, 'r')
    fgrid   = h5py.File(grid_fname, 'r')

    print(fmodel['model'].keys())

    vel_inv  = np.array(fmodel['model']["vel_inv_%s"%(Nstep)])
    xi_inv  = np.array(fmodel['model']["xi_inv_%s"%(Nstep)])
    eta_inv  = np.array(fmodel['model']["eta_inv_%s"%(Nstep)])

    grid_r = np.array(fgrid["Mesh"]["node_coords_r"])
    grid_t = np.array(fgrid["Mesh"]["node_coords_t"])
    grid_p = np.array(fgrid["Mesh"]["node_coords_p"])
    
    fmodel.close()
    fgrid.close()

    model={}
    dir_lat={};dir_lon={};dir_dep={}
    np_lat=[];np_lon=[];np_dep=[]
    for i in range(len(vel_inv)):
        lat  = '%.8g'%(grid_t[i]); 
        lon  = '%.8g'%(grid_p[i]); 
        dep  = '%.8g'%(R_earth - grid_r[i]); 
        vel = vel_inv[i]
        xi   = xi_inv[i]
        eta  = eta_inv[i]
        sigma = vel_inv[i] 

        model['%s_%s_%s'%(lat,lon,dep)]=(vel,xi,eta,sigma)
        dir_lat[lat] = float(lat)
        dir_lon[lon] = float(lon)
        dir_dep[dep] = float(dep)



    for i in dir_lat:
        np_lat.append(dir_lat[i])
    np_lat.sort()
    for i in dir_lon:
        np_lon.append(dir_lon[i])
    np_lon.sort()
    for i in dir_dep:
        np_dep.append(dir_dep[i])
    np_dep.sort()



    all_lat=np.linspace(stlat,edlat,Nlat,dtype=float)
    all_lon=np.linspace(stlon,edlon,Nlon,dtype=float)



    np_lat_id=[];np_lon_id=[];np_dep_id=[]

    np_lat_id = relative_id_nosort(all_lat,np_lat) 
    np_lon_id = relative_id_nosort(all_lon,np_lon) 
    np_dep_id = relative_id_nosort(all_dep,np_dep) 

    target_dep_value = np.full((len(all_dep),Nlat*Nlon,7),np.nan)    # dep,lat,lon,vel,xi,eta,simga
    for idep in range(0,len(all_dep)):
        dep=all_dep[idep]

        for i in range(0,Nlat):
            lat=all_lat[i]
            
            for j in range(0,Nlon):
                lon=all_lon[j]

                (phy_lat, phy_lon) = (lat,lon)


                #try:

                # 确定了目标点，开始寻找周围八个点
                lat1=np_lat[np_lat_id[i]];lat2=np_lat[np_lat_id[i]+1]
                lon1=np_lon[np_lon_id[j]];lon2=np_lon[np_lon_id[j]+1]
                dep1=np_dep[np_dep_id[idep]];dep2=np_dep[np_dep_id[idep]+1]

                if (not(lat>=lat1 and lat<lat2 and lon>=lon1 and lon<lon2 and dep>=dep1 and dep<dep2)):
                    target_dep_value[idep,i*Nlon+j,:] = (dep,lat,lon,np.nan,np.nan,np.nan,np.nan)
                    # print(lat,lon,dep,"out of region")
                else:
                    # 三个方向比例
                    rdep=(dep-dep1)/(dep2-dep1); rdep=min(rdep,1);rdep=max(0,rdep)
                    rlat=(lat-lat1)/(lat2-lat1); rlat=min(rlat,1);rlat=max(0,rlat)
                    rlon=(lon-lon1)/(lon2-lon1); rlon=min(rlon,1);rlon=max(0,rlon)

                    vel=0
                    vel+=rlat*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep2)][0]
                    vel+=rlat*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep1)][0]
                    vel+=rlat*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep2)][0]
                    vel+=rlat*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep1)][0]
                    vel+=(1-rlat)*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep2)][0]
                    vel+=(1-rlat)*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep1)][0]
                    vel+=(1-rlat)*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep2)][0]
                    vel+=(1-rlat)*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep1)][0]


                    xi=0
                    xi+=rlat*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep2)][1]
                    xi+=rlat*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep1)][1]
                    xi+=rlat*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep2)][1]
                    xi+=rlat*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep1)][1]
                    xi+=(1-rlat)*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep2)][1]
                    xi+=(1-rlat)*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep1)][1]
                    xi+=(1-rlat)*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep2)][1]
                    xi+=(1-rlat)*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep1)][1]


                    eta=0
                    eta+=rlat*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep2)][2]
                    eta+=rlat*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep1)][2]
                    eta+=rlat*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep2)][2]
                    eta+=rlat*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep1)][2]
                    eta+=(1-rlat)*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep2)][2]
                    eta+=(1-rlat)*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep1)][2]
                    eta+=(1-rlat)*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep2)][2]
                    eta+=(1-rlat)*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep1)][2]

                    sigma = 0
                    sigma+=rlat*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep2)][3]
                    sigma+=rlat*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon2,dep1)][3]
                    sigma+=rlat*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep2)][3]
                    sigma+=rlat*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat2,lon1,dep1)][3]
                    sigma+=(1-rlat)*rlon*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep2)][3]
                    sigma+=(1-rlat)*rlon*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon2,dep1)][3]
                    sigma+=(1-rlat)*(1-rlon)*rdep*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep2)][3]
                    sigma+=(1-rlat)*(1-rlon)*(1-rdep)*model['%.8g_%.8g_%.8g'%(lat1,lon1,dep1)][3]

                    target_dep_value[idep,i*Nlon+j,:] = (dep,phy_lat,phy_lon,vel,xi,eta,sigma)
    return target_dep_value


In [9]:
def ave_model(target_model):
    Ndep  = np.size(target_model,0)
    Ngrid = np.size(target_model,1)
    average_model = np.copy(target_model)    # dep,lat,lon,slowness,xi,eta,vel
    
    for idep in range(Ndep):
        ave_value = 0
        count = 0
        for igrid in range(Ngrid):
            if(np.isnan(target_model[idep,igrid,3])):
                # nan, 
                pass
            else:
                print(target_model[idep,igrid,:])
                print(ave_value,count)
                ave_value = ave_value + target_model[idep,igrid,3]
                count = count + 1
        
        ave_value = ave_value/count

        for igrid in range(Ngrid):
            if(np.isnan(target_model[idep,igrid,3])):
                # nan, 
                pass
            else:
                average_model[idep,igrid,3] = ave_value

    return average_model

In [10]:

stlat=30.0
edlat=36.0
stlon=30.0
edlon=36.0

# 数据区域范围
data_stlat = stlat
data_edlat = edlat
data_stlon = stlon
data_edlon = edlon

# 投影方式
projection = "M11c"

# 离散点数
Nlat=101
Nlon=101

# 画图指定深度
all_dep = [10,30]


print('loading file ...')
# 读取模型文件

# test_path = "ckb_N3_10_6"
#test_path = "t5_ckb_N3_8_4_增加各向异性+t4"




# ckb model
# test_path = "OUTPUT_FILES_swap_inv_abs"
# target_fname = "OUTPUT_FILES/OUTPUT_FILES_swap_signal/out_data_sim_group_0.h5"   # 检测板反演的最终模型
# target_Nstep = "0001"
# ref_fname = "OUTPUT_FILES/OUTPUT_FILES_swap_inv_abs/out_data_sim_group_0.h5"  # 检测板反演的初始模型
# ref_Nstep = "0001"
# grid_fname = "OUTPUT_FILES/OUTPUT_FILES_swap_signal/out_data_grid.h5"                # 网格文件
# output = "%s_%s"%(test_path,target_Nstep)


# ckb inv model
test_path = "OUTPUT_FILES_swap_inv_abs"
target_fname = "OUTPUT_FILES/%s/out_data_sim_group_0.h5"%(test_path)   # 检测板反演的最终模型
target_Nstep = "0002"
ref_fname = "OUTPUT_FILES/%s/out_data_sim_group_0.h5"%(test_path)  # 检测板反演的初始模型
ref_Nstep = "0001"
grid_fname = "OUTPUT_FILES/%s/out_data_grid.h5"%(test_path)                # 网格文件
output = "%s_%s"%(test_path,target_Nstep)


# 插值获得画图数据
target_model = read_model_file_horizontal(grid_fname,target_fname,target_Nstep,data_stlat,data_edlat,data_stlon,data_edlon,Nlat,Nlon,all_dep)
ref_model = read_model_file_horizontal(grid_fname,ref_fname,ref_Nstep,data_stlat,data_edlat,data_stlon,data_edlon,Nlat,Nlon,all_dep)     # initiai model
# ref_model = ave_model(target_model)




loading file ...
<KeysViewHDF5 ['eta_inv_0001', 'eta_inv_0002', 'eta_inv_0003', 'eta_inv_0004', 'eta_inv_0005', 'vel_inv_0001', 'vel_inv_0002', 'vel_inv_0003', 'vel_inv_0004', 'vel_inv_0005', 'xi_inv_0001', 'xi_inv_0002', 'xi_inv_0003', 'xi_inv_0004', 'xi_inv_0005']>
<KeysViewHDF5 ['eta_inv_0001', 'eta_inv_0002', 'eta_inv_0003', 'eta_inv_0004', 'eta_inv_0005', 'vel_inv_0001', 'vel_inv_0002', 'vel_inv_0003', 'vel_inv_0004', 'vel_inv_0005', 'xi_inv_0001', 'xi_inv_0002', 'xi_inv_0003', 'xi_inv_0004', 'xi_inv_0005']>


In [16]:
# test_path = "OUTPUT_FILES_swap_inv_abs"
# target_fname = "OUTPUT_FILES/%s/out_data_sim_group_0.h5"%(test_path)   # 检测板反演的最终模型
# target_Nstep = "0002"
# ref_fname = "OUTPUT_FILES/%s/out_data_sim_group_0.h5"%(test_path)  # 检测板反演的初始模型
# ref_Nstep = "0001"
# grid_fname = "OUTPUT_FILES/%s/out_data_grid.h5"%(test_path)                # 网格文件
# output = "%s_%s"%(test_path,target_Nstep)  
R_earth = 6371

test_path = "OUTPUT_FILES_swap_inv_abs"
model_fname = "OUTPUT_FILES/%s/out_data_sim_group_0.h5"%(test_path)
grid_fname  = "OUTPUT_FILES/%s/out_data_grid.h5"%(test_path)
fmodel  = h5py.File(model_fname, 'r')
fgrid   = h5py.File(grid_fname, 'r')

vel_inv1  = np.array(fmodel['model']["vel_inv_0001"])
xi_inv1  = np.array(fmodel['model']["xi_inv_0001"])
eta_inv1  = np.array(fmodel['model']["eta_inv_0001"])

vel_inv2  = np.array(fmodel['model']["vel_inv_0002"])
xi_inv2  = np.array(fmodel['model']["xi_inv_0002"])
eta_inv2  = np.array(fmodel['model']["eta_inv_0002"])

grid_r = np.array(fgrid["Mesh"]["node_coords_r"])
grid_t = np.array(fgrid["Mesh"]["node_coords_t"])
grid_p = np.array(fgrid["Mesh"]["node_coords_p"])

fmodel.close()
fgrid.close()

model={}
dir_lat={};dir_lon={};dir_dep={}
np_lat=[];np_lon=[];np_dep=[]
for i in range(len(vel_inv1)):
    lat  = '%.8g'%(grid_t[i]); 
    lon  = '%.8g'%(grid_p[i]); 
    dep  = '%.8g'%(R_earth - grid_r[i]); 
    vel = (vel_inv1[i]-vel_inv2[i])/vel_inv1[i]

    # model['%s_%s_%s'%(lat,lon,dep)]=(vel)
    if(i > 113490 - 100 and i < 113490 + 100):
        print('lat: %8.4f, lon: %8.4f, dep: %8.4f, vel: %6.4f'%(float(lat),float(lon),float(dep),vel))



lat:  30.9333, lon:  31.7667, dep:  20.0000, vel: 0.0052
lat:  30.9333, lon:  31.8000, dep:  20.0000, vel: 0.0049
lat:  30.9333, lon:  31.8333, dep:  20.0000, vel: 0.0046
lat:  30.9333, lon:  31.8667, dep:  20.0000, vel: 0.0044
lat:  30.9333, lon:  31.9000, dep:  20.0000, vel: 0.0041
lat:  30.9333, lon:  31.9333, dep:  20.0000, vel: 0.0038
lat:  30.9333, lon:  31.9667, dep:  20.0000, vel: 0.0036
lat:  30.9333, lon:  32.0000, dep:  20.0000, vel: 0.0033
lat:  30.9667, lon:  30.0000, dep:  20.0000, vel: 0.0035
lat:  30.9667, lon:  30.0333, dep:  20.0000, vel: 0.0037
lat:  30.9667, lon:  30.0667, dep:  20.0000, vel: 0.0040
lat:  30.9667, lon:  30.1000, dep:  20.0000, vel: 0.0043
lat:  30.9667, lon:  30.1333, dep:  20.0000, vel: 0.0046
lat:  30.9667, lon:  30.1667, dep:  20.0000, vel: 0.0048
lat:  30.9667, lon:  30.2000, dep:  20.0000, vel: 0.0051
lat:  30.9667, lon:  30.2333, dep:  20.0000, vel: 0.0054
lat:  30.9667, lon:  30.2667, dep:  20.0000, vel: 0.0057
lat:  30.9667, lon:  30.3000, d

In [14]:
pert = (vel_inv1-vel_inv2)/vel_inv1

print(np.argmax(pert))

print()

113490
